52 million mothers in the United States live with their children.
Of those, over 4.5 million are in poverty.

That poverty rate of 9.0 percent is a bit lower than the overall poverty rate of 9.3 percent,[^spm] largely thanks to safety net programs that target families with children.
Yet, many of those programs fail to reach low-income mothers due to qualification rules and complex bureaucracies.

[^spm]: This analysis uses the 2020 Current Population Survey March Supplement, and the [Supplemental Poverty Measure](https://www.census.gov/library/publications/2021/demo/p60-275.html).
The official SPM poverty rate was 9.1 percent in 2020, but the public use microdata that powers our analysis produces slightly different results.

A simpler alternative, which [other countries adopt](https://child-allowance.ubicenter.org/policies.html#other-countries) and [which we've written about in several other reports](https://www.ubicenter.org/tag/child-allowance/), is a universal child allowance: a flat amount given each month to parents for each of their children.

While child allowances chiefly reduce child poverty, they also lift mothers out of poverty.
For example, a \$200 per month child allowance would lift one million mothers out of poverty, at an annual cost of \$173 billion.

In [1]:
import plotly.express as px
from ubicenter import format_fig
from openfisca_us import Microsimulation

sim = Microsimulation()

In [2]:
# sim.calc("is_mother").sum()  # 52 million.
# sim.calc("spm_unit_is_in_spm_poverty", map_to="person").mean()  # 9.3%

In [3]:
from openfisca_us.api import *

from openfisca_us.model_api import *


class spm_unit_net_income(Variable):
    value_type = float
    entity = SPMUnit
    label = "Net income"
    definition_period = YEAR
    unit = USD

    formula = sum_of_variables(
        [
            "spm_unit_net_income_reported", "child_allowance"
        ]
    )


def get_mother_poverty(child_allowance_amount):
    
    class child_allowance(Variable):
        value_type = float
        entity = Person
        definition_period = YEAR

        def formula(person, period, parameters):
            return person("is_child", period) * child_allowance_amount * 12


    class add_child_allowance(Reform):
        def apply(self):
            self.update_variable(child_allowance)
            self.update_variable(spm_unit_net_income)

    reformed = Microsimulation(add_child_allowance)

    dfr = reformed.df(["is_mother", "spm_unit_is_in_spm_poverty"], map_to="person")
    return pd.Series(dict(
        child_allowance=i,
        mother_poverty_count=dfr[dfr.is_mother].spm_unit_is_in_spm_poverty.sum(),
        cost=reformed.calc("child_allowance").sum()
    ))


In [4]:
l = []

for i in np.arange(0, 500, 10):
    l.append(get_mother_poverty(i))

df = pd.DataFrame(l)

In [5]:
df["hover"] = (
    "A $" + df.child_allowance.astype(int).astype(str) +
    " monthly child allowance would cost $" +
    (df.cost / 1e9).astype(int).astype(str) +
    " billion per year and lift " +
    ((df.mother_poverty_count[0] - df.mother_poverty_count) / 1e6).round(1).astype(str) + 
    " million mothers out of poverty."
    )
LABELS = dict(child_allowance="Monthly child allowance", mother_poverty_count="Mothers in poverty")
HOVER_TEMPLATE = "%{customdata[0]}<extra></extra>"
fig = px.line(
    df, x="child_allowance", y="mother_poverty_count", labels=LABELS, custom_data=["hover"],
    title="How a monthly child allowance would lift moms out of poverty")
fig.update_layout(yaxis_range=[0, 5e6], xaxis_tickformat="$0")
fig.update_traces(hovertemplate=HOVER_TEMPLATE)
format_fig(fig)

Maternal poverty is associated with [depression and worsened child development](https://srcd.onlinelibrary.wiley.com/doi/abs/10.1111/1467-8624.00379).
Mothers spend about [1.7 hours per day](https://www.bls.gov/charts/american-time-use/activity-by-parent.htm) caring for children.
And of course, mothers suffer from the consequences of poverty like any other person does.
Child allowances can improve child outcomes, compensate parents for raising the next generation, and improve the well-being of moms everywhere.